In [1]:
# Install a pip package in the current Jupyter kernel - Uncomment if needed
# import sys
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install requests
# !{sys.executable} -m pip install bs4
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install selenium
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install urllib
# !{sys.executable} -m pip install re
# !{sys.executable} -m pip install nbimporter

In [2]:
# Import the relevant packages
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np
from urllib.parse import urljoin
import re
import nbimporter

# import functions from different scraping_util notebook; Make sure both scripts are in the same folder
%run scraping_util.ipynb


In [3]:
# import data and filter for relevant BL
data = pd.read_csv(r'C:\Users\mariu\Documents\pol_viol\pol_viol\liste_gemeinden_ger.csv', delimiter=';')
data = data.dropna(thresh=8).reset_index(drop=True)
data['AGS'] = '0' + data['Land'].astype(str) + data['RB'].astype(int).astype(str) + data['Kreis'].astype(int).astype(str) + '0'+ data['Gem'].astype(int).astype(str)
data.loc[data['Gem'] < 10, 'AGS'] = '0' + data['Land'].astype(str) + data['RB'].astype(int).astype(str) + data['Kreis'].astype(int).astype(str) + '00' + data['Gem'].astype(int).astype(str)
data_hessen= data[data['Land'] == 6]
data_scrape = data_hessen[['Gemeindename', 'plz', 'AGS']]
data_scrape.head()

,Gemeindename,plz,AGS
2474,Darmstadt,64283.0,06411000
2475,Frankfurt am Main,60311.0,06412000
2476,Offenbach am Main,63065.0,06413000
2477,Wiesbaden,65183.0,06414000
2478,Abtsteinach,69518.0,06431001


In [4]:
# set up webdriver and search field element
# regularly check for the right version of webdriver
# chrome version: 97.0.4692.99
# chrome driver 97.0.4692.71
# Important to keep in mind: both need to be of version 97.
d=webdriver.Chrome(r"C:\Users\mariu\Documents\pol_viol\pol_viol\chromedriver.exe")
d.get('https://wahlen.votemanager.de/#')
e = d.find_element_by_id('suchfeld')

# search for each municipality and get the url to the municipality specific overview page
data_scrape['url'] = data_scrape['Gemeindename'].apply(get_url1)
data_scrape.reset_index(drop=True, inplace=True)
data_scrape = data_scrape[data_scrape.astype(str)['url']!='[]'].reset_index(drop=True)

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [7]:
data_scrape['results'] = data_scrape['url'].str.len()

In [8]:
# Find the relevant link and ignore mismatches
for row in data_scrape.iterrows():
    if row[1][4] > 1:
        for link in row[1][3]:
            if row[1][2] in link:
                data_scrape.loc[(data_scrape['Gemeindename'] == row[1][0]), 'url'] = link

In [9]:
# Create links for the three last election cycles
data_scrape['url_copy'] = data_scrape['url'].astype(str)
data_exp = data_scrape['url_copy'].str.replace("\['", "").str.replace("'\]", "").str.split('/',expand=True)
data_scrape['url_21'] = data_exp[0]+ '//' + data_exp[2]+ '/' + '2021-03-14/' + data_exp[3] + '/html5/' + data_exp[4]
data_scrape['url_16'] = data_exp[0]+ '//' + data_exp[2]+ '/' + '20160306/' + data_exp[3] + '/html5/' + data_exp[4]
data_scrape['url_11'] = data_exp[0]+ '//' + data_exp[2]+ '/' + '20110327/' + data_exp[3] + '/html5/' + data_exp[4]

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
def get_url3(url2):
    """ Function to get three possible links for every election. Each of 'matches' defines a possible link 
        specification. Only input is the link column.
    """
    
    r = requests.get(url2) 
    soup = BeautifulSoup(r.text)
    base = url2
    hrefs = [link.get('href') for link in soup.find_all('a')]
    
    matches = ['Stadtverordnetenwahl_Hessen', 'Gemeindewahl_Hessen', 'Gemeindewahl-Hessen']
    
    href_2=[href for href in hrefs if any(x in href for x in matches)]
    
    if href_2:
        s = href_2.pop()
        return urljoin(base,s)
    else:
        return np.nan

In [19]:
# Some invalid links -> get rid of them
data_scrape = data_scrape[~data_scrape['url_21'].isna()]

In [20]:
data_scrape['url_21'] = data_scrape['url_21'].apply(get_url3)
data_scrape['url_16'] = data_scrape['url_16'].apply(get_url3)
data_scrape['url_11'] = data_scrape['url_11'].apply(get_url3)

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [21]:
data_21 = scrape_webpage(data_scrape, 'Gemeindename', 'url_21', 2021)
data_16 = scrape_webpage(data_scrape, 'Gemeindename', 'url_16', 2016)
data_11 = scrape_webpage(data_scrape, 'Gemeindename', 'url_11', 2011)

City 0/416


C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.


City 1/416


C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.


City 2/416


...

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.


City 386/416
City 387/416


C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.


City 388/416
City 389/416


C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.


In [22]:
data_hessen = data_21.append(data_16)
data_hessen = data_hessen.append(data_11)
data_hessen.reset_index(drop=True, inplace=True)
data_hessen['candidates_name'] = data_hessen['candidate'].str.split(',', expand=True)[0]
data_hessen['party'] = data_hessen['candidate'].str.split(',', expand=True)[1]

In [23]:
data_hessen = data_hessen[~data_hessen['party'].isna()]
data_hessen['candidates_name'] = data_hessen['candidates_name'].str.replace('Prof ', '').str.replace('habil ', '').str.replace('Dr med ', '').str.replace('Dr Dr ', '').str.replace('Dr ', '').str.replace('dent ', '').str.replace('rer nat ', '').str.replace('PD ', '')
data_hessen['first_name'] = data_hessen['candidates_name'].str.split(' ', expand=True)[0]
data_hessen['BL'] = 'HE'
data_hessen_2 = data_hessen[(data_hessen['party'].str.contains('CDU')) | (data_hessen['party'].str.contains('SPD')) | (data_hessen['party'].str.contains('AfD')) | (data_hessen['party'].str.contains('GRUNE')) | ((data_hessen['party'].str.contains('FDP'))) | ((data_hessen['party'].str.contains('LINKE')))]
data_hessen_2.reset_index(drop=True, inplace=True)
data_hessen_2

,candidate,city,year,candidates_name,party,first_name,BL
0,"Dr Nils Koessler, CDU",Frankfurt am Main,2021,Nils Koessler,CDU,Nils,HE
1,"Stephan Siegler, CDU",Frankfurt am Main,2021,Stephan Siegler,CDU,Stephan,HE
2,"Christiane Loizides, CDU",Frankfurt am Main,2021,Christiane Loizides,CDU,Christiane,HE
3,"Sabine Fischer, CDU",Frankfurt am Main,2021,Sabine Fischer,CDU,Sabine,HE
4,"Martin-Benedikt Schaefer, CDU",Frankfurt am Main,2021,Martin-Benedikt Schaefer,CDU,Martin-Benedikt,HE
...,...,...,...,...,...,...,...
81195,"Hans-Juergen Mueller, GRUNE",Gemünden (Wohra),2011,Hans-Juergen Mueller,GRUNE,Hans-Juergen,HE
81196,"Holger Pflueger-Grone, GRUNE",Gemünden (Wohra),2011,Holger Pflueger-Grone,GRUNE,Holger,HE
81197,"Nicolette Weiss, GRUNE",Gemünden (Wohra),2011,Nicolette Weiss,GRUNE,Nicolette,HE
81198,"Sabine Jacobs, GRUNE",Gemünden (Wohra),2011,Sabine Jacobs,GRUNE,Sabine,HE


In [24]:
# Potential new observations for analysis
count = 0
data_hessen_3 = data_hessen_2[data_hessen_2['year'] != 2011]
for city in data_hessen_3['city'].unique():
    count += len(data_hessen_3[data_hessen_3['city'] == city]['party'].unique())
count

1471

In [25]:
# export
data_hessen_2.to_csv(r'C:\Users\mariu\Documents\pol_viol\pol_viol\scraped_hessen.csv')